<a href="https://colab.research.google.com/github/abakashi/dinamicas_dex_days/blob/main/Wrangling_Case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
#Import das bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [160]:
#Carregamento dos raw datasets nos dataframes
rwdf_churn = pd.read_csv('churn.csv', dtype={'employee': int, 'numVotes': int, 'stillExists': bool})
rwdf_votes = pd.read_csv('votes.csv')
rwdf_cmm_int = pd.read_csv('commentInteractions.csv', dtype={'employee': int, 'liked': str, 'disliked': str})
rwdf_cmm_clean = pd.read_csv('comments_clean_anonimized.csv')

In [161]:
#Começando o estudo do Churn

# Criando uma função para colocar as datas num formato mais palatável.
def time_formatter(series):
  dates = list(series)
  tmz = list()
  dts = list()

  for date in dates:
    if date[-7] == 'S':
      day = date[8:10]
      mon = date[4:7]
      yr = date[-4:]
      time = date[11:19]
      dts.append(f'{day}/{mon}/{yr} - {time}')
      tmz.append('CEST')
    else:
      day = date[8:10]
      mon = date[4:7]
      yr = date[-4:]
      time = date[11:19]
      dts.append(f'{day}/{mon}/{yr} - {time}')
      tmz.append('CET')
  return (pd.Series(dts),pd.Series(tmz))

#criando um novo dataset vazio
rwdf_churn_date_form = pd.DataFrame()

#implementando as duas colunas referentes à data
rwdf_churn_date_form['LPDate_formated'], rwdf_churn_date_form['LPDate_tmz'] = time_formatter(rwdf_churn['lastParticipationDate'])

#convertendo a coluna para o formato datetime.
rwdf_churn_date_form['LPDate_formated'] = pd.to_datetime(rwdf_churn_date_form['LPDate_formated'], format='%d/%b/%Y -  %H:%M:%S')

#criando as demais colunas no novo dataset
rwdf_churn_date_form['employee'], rwdf_churn_date_form['com_alias'], rwdf_churn_date_form['votes'], rwdf_churn_date_form['still_exists']  = rwdf_churn['employee'], rwdf_churn['companyAlias'], rwdf_churn['numVotes'], rwdf_churn['stillExists']

#adicionando + uma hora de delta nos timezones CEST (horario de verão europeu)
rwdf_churn_date_form.loc[rwdf_churn_date_form['LPDate_tmz'] == 'CEST', 'LPDate_formated'] = rwdf_churn_date_form['LPDate_formated'] + pd.Timedelta('1 hour')

#associando o conjunto novo com o antigo
rwdf_churn = rwdf_churn_date_form

In [169]:
#checando por valores nulos
rwdf_churn.isna().sum()
#nenhum valor nulo

#checando valores duplicados
rwdf_churn.duplicated().sum()
#sem linhas duplicadas

rwdf_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   LPDate_formated  4847 non-null   datetime64[ns]
 1   LPDate_tmz       4847 non-null   object        
 2   employee         4847 non-null   int64         
 3   com_alias        4847 non-null   object        
 4   votes            4847 non-null   int64         
 5   still_exists     4847 non-null   bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(2)
memory usage: 194.2+ KB


In [174]:
#Analisando o conjunto votes

#Aplicando a time_formatter nas datas
rwdf_votes['vDate_form'], rwdf_votes['vDate_tmz'] = time_formatter(rwdf_votes['voteDate'])

#convertendo para formato datetime
rwdf_votes['vDate_form'] = pd.to_datetime(rwdf_votes['vDate_form'], format='%d/%b/%Y -  %H:%M:%S')

#adicionando mais uma hora nos horários em horário de verão
rwdf_votes.loc[rwdf_votes['vDate_tmz'] == 'CEST', 'vDate_form'] = rwdf_votes['vDate_form'] + pd.Timedelta('1 hour')

#apagando a coluna de data antiga
rwdf_votes.drop('voteDate', axis=1, inplace=True)

In [180]:
#Verificando valores nulos: Sem valores nulos
rwdf_votes.isna().sum()


rwdf_votes.duplicated().sum()

rwdf_votes

,employee,companyAlias,vote,vDate_form,vDate_tmz
0,31,56aec740f1ef260003e307d6,4,2016-02-01 01:00:00,CET
1,33,56aec740f1ef260003e307d6,4,2016-02-01 01:00:00,CET
2,79,56aec740f1ef260003e307d6,4,2016-02-01 01:00:00,CET
3,94,56aec740f1ef260003e307d6,4,2016-02-01 01:00:00,CET
4,16,56aec740f1ef260003e307d6,2,2016-02-01 01:00:00,CET
...,...,...,...,...,...
221227,17,573a0671b5ec330003add34a,3,2016-06-03 03:00:00,CEST
221228,17,573a0671b5ec330003add34a,4,2016-06-07 03:00:00,CEST
221229,17,573a0671b5ec330003add34a,4,2016-06-13 03:00:00,CEST
221230,17,573a0671b5ec330003add34a,4,2016-06-28 03:00:00,CEST
